In [ ]:
from nmnh_ms_tools.records import Reference
from xmu import EMuAPI, EMuRecord, is_ref, group_columns

In [ ]:
# FIXME: These functions should ultimately be incorporated into EMuAPI

def get_irn(col, rec, api):
    """Gets the IRN for an attached record"""
    try:
        return rec["irn"]
    except KeyError:
        field_info = EMuRecord.schema.get_field_info("ebibliography", col)
        filter_ = {k: v if v else r"\!\+" for k, v in rec.items()}
        resp = api.search(field_info["RefTable"], filter_=filter_).send()
        if not resp.hits:
            resp = api.insert(field_info["RefTable"], rec).send()
        return resp.first()["irn"]


def get_irns(rec, api):
    """Gets IRNs for attached records"""
    for col, val in rec.items():
        if is_ref(col):
            if isinstance(val, list):
                vals = []
                for row in val:
                    vals.append(get_irn(col, get_irns(row, api), api))
                rec[col] = vals
            else:
                rec[col] = get_irn(col, get_irns(row, api), api)
    return rec


def prep_insert(rec, module, api):
    """Prepares a record to be inserted into EMu using the API"""
    return group_columns(get_irns(rec, api), module=module)

In [ ]:
# Create the EMuAPI object based on params in the TOML file
api = EMuAPI(config_path=r"emurestapi.toml")

# Resolve the DOI
ref = Reference("https://doi.org/10.1093/petrology/egu024")

# HACK: Delete nested fields from record
rec = ref.to_emu().to_dict()
del rec["NteAttributedToRef_nesttab"]

# Create then display the bibliography record
resp = api.insert(
    "ebibliography", prep_insert(rec, "ebibliography", api)
)
resp.first()